In [31]:
import os
import json
import pandas as pd

In [32]:
dataset_grid_res = {x.split('_')[0]: x for x in os.listdir('grid_search_results') if
                    x.endswith('.csv') and 'process' not in x}
dataset_grid_res

{'accent': 'accent_grid_val.csv',
 'ecoli': 'ecoli_grid_val.csv',
 'iris': 'iris_grid_val.csv'}

In [33]:
os.makedirs("param_files", exist_ok=True)

In [34]:
df = pd.read_csv(os.path.join('grid_search_results', list(dataset_grid_res.values())[0]))
df.head()

,dataset,conf_num,reg_type,weight_type,alpha,use_layer_norm,use_aug,lr,batch_size,train_full_score,train_partial_score,val_full_score,val_partial_score
0,Accent,1,l1,None,0.0,True,True,0.001,32,0.760806,0.726 +- 0.025,0.792453,0.756 +- 0.026
1,Accent,2,l1,None,0.0,True,True,0.001,64,0.783883,0.74 +- 0.031,0.735849,0.725 +- 0.038
2,Accent,3,l1,None,0.0,True,True,0.010,32,0.817582,0.763 +- 0.03,0.792453,0.756 +- 0.043
3,Accent,4,l1,None,0.0,True,True,0.010,64,0.839194,0.783 +- 0.026,0.886792,0.818 +- 0.051
4,Accent,5,l1,None,0.0,True,True,0.100,32,0.717216,0.681 +- 0.02,0.773585,0.753 +- 0.018


In [35]:
for dataset, grid_res in dataset_grid_res.items():
    df = pd.read_csv(os.path.join('grid_search_results', grid_res))
    df["train_partial_scores_mean"] = df["train_partial_score"].apply(lambda x: float(x.split(" +- ")[0]))
    df["val_partial_scores_mean"] = df["val_partial_score"].apply(lambda x: float(x.split(" +- ")[0]))

    df["full_score"] = df[["train_full_score", "val_full_score"]].min(axis=1)
    df["partial_scores_mean"] = df[["train_partial_scores_mean", "val_partial_scores_mean"]].min(axis=1)

    df["diff_full_partial"] = df["full_score"] - df["partial_scores_mean"]

    df.sort_values(by="diff_full_partial", ascending=True, inplace=True, kind='stable')
    df.sort_values(by="partial_scores_mean", ascending=False, inplace=True, kind='stable')
    df.to_csv(os.path.join("param_files", f"{dataset}_params.csv"), index=False)


    # create params dict of reg_type	weight_type	alpha	use_layer_norm	use_aug	lr	batch_size

    params = {
        "reg_type": str(df.iloc[0]["reg_type"]),
        "weight_type": str(df.iloc[0]["weight_type"]),
        "alpha": float(df.iloc[0]["alpha"]),
        "use_layer_norm": bool(df.iloc[0]["use_layer_norm"]),
        "use_aug": bool(df.iloc[0]["use_aug"]),
        "lr": float(df.iloc[0]["lr"]),
        "batch_size": int(df.iloc[0]["batch_size"]),
    }

    with open(os.path.join("param_files", f"{dataset}_params.json"), 'w') as f:
        json.dump(params, f)


In [36]:
for i, dataset in enumerate(os.listdir("param_files")):
    if dataset.endswith(".json"):
        f = open("removal_test_commands", 'w' if i == 0 else 'a')
        dataset = dataset.split("_")[0]
        f.write(f"python main_experimens.py --dataset {dataset.capitalize()} --run_grid False --run_ts False\n")

In [37]:
# load all params files to one df

df = pd.DataFrame()

for dataset in os.listdir("param_files"):
    if dataset.endswith(".json"):
        dataset = dataset.split("_")[0]

        if dataset.lower() in ['wifi', 'wine']:
            continue

        with open(os.path.join("param_files", dataset + "_params.json"), 'r') as f:
            params = json.load(f)
        df = df.append(pd.DataFrame(params, index=[dataset]))

df

,reg_type,weight_type,alpha,use_layer_norm,use_aug,lr,batch_size
ecoli,max,mult,0.5,True,True,0.01,32
iris,l2,None,1.0,True,False,0.10,32
accent,l2,avg,0.5,True,True,0.01,64
